**Test for NYC trees assignment**

To start I use download.py to get a smaller dataset (only 10000 instead of the original with 100000 rows)
My new dataset is data_10000.csv
(see second cell where I read the data, 10000 values takes 66 ms, while 100000 values takes 588)

In [7]:
#Import the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
#Open & check the data

trees = pd.read_csv("data_10000.csv")
##%timeit 66 ms ± 3.95 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

#%timeit trees = pd.read_csv("data_100000.csv")
#588 ms ± 8.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
trees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 42 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   created_at  10000 non-null  object 
 1   tree_id     10000 non-null  int64  
 2   block_id    10000 non-null  int64  
 3   the_geom    10000 non-null  object 
 4   tree_dbh    10000 non-null  int64  
 5   stump_diam  10000 non-null  int64  
 6   curb_loc    10000 non-null  object 
 7   status      10000 non-null  object 
 8   health      9415 non-null   object 
 9   spc_latin   9415 non-null   object 
 10  spc_common  9415 non-null   object 
 11  steward     9415 non-null   object 
 12  guards      9415 non-null   object 
 13  sidewalk    9415 non-null   object 
 14  user_type   10000 non-null  object 
 15  problems    9415 non-null   object 
 16  root_stone  10000 non-null  object 
 17  root_grate  10000 non-null  object 
 18  root_other  10000 non-null  object 
 19  trnk_wire   10000 non-null

Seeing that, things to do:

0 created at : it should be a date...

In [9]:
trees.head(6)

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,08/27/2015,180683,348711,POINT (-73.84421521958048 40.723091773924274),3,0,OnCurb,Alive,Fair,Acer rubrum,...,28,16,QN17,Forest Hills,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.768749
1,09/03/2015,200540,315986,POINT (-73.81867945834878 40.79411066708779),21,0,OnCurb,Alive,Fair,Quercus palustris,...,27,11,QN49,Whitestone,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.837379
2,09/05/2015,204026,218365,POINT (-73.93660770459083 40.717580740099116),3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,50,18,BK90,East Williamsburg,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.891267
3,09/05/2015,204337,217969,POINT (-73.93445615919741 40.713537494833226),10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,53,18,BK90,East Williamsburg,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.253136
4,08/30/2015,189565,223043,POINT (-73.97597938483258 40.66677775537875),21,0,OnCurb,Alive,Good,Tilia americana,...,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.425999
5,08/30/2015,190422,106099,POINT (-73.98494997200308 40.770045625891846),11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,67,27,MN14,Lincoln Square,1014500,New York,40.770046,-73.984950,9.884187e+05,219825.522669


In [31]:
trees.shape

(10000, 42)

In order to get some insight about the data, I check the website of the dataset:

https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh



In [15]:
trees.isna().sum() #useless, the columns with nan are the ones that are objects type!

created_at      0
tree_id         0
block_id        0
the_geom        0
tree_dbh        0
stump_diam      0
curb_loc        0
status          0
health        585
spc_latin     585
spc_common    585
steward       585
guards        585
sidewalk      585
user_type       0
problems      585
root_stone      0
root_grate      0
root_other      0
trnk_wire       0
trnk_light      0
trnk_other      0
brnch_ligh      0
brnch_shoe      0
brnch_othe      0
address         0
zipcode         0
zip_city        0
cb_num          0
borocode        0
boroname        0
cncldist        0
st_assem        0
st_senate       0
nta             0
nta_name        0
boro_ct         0
state           0
latitude        0
longitude       0
x_sp            0
y_sp            0
dtype: int64

In [24]:
trees.zipcode.value_counts()

10306    448
11105    355
11375    263
10457    250
11215    247
        ... 
11360      1
11004      1
10035      1
10469      1
10115      1
Name: zipcode, Length: 160, dtype: int64

In [27]:
trees[trees.columns[10]].unique() #thus probably with regex, change the names, all with no capital letters

array(['red maple', 'pin oak', 'honeylocust', 'American linden',
       'London planetree', 'ginkgo', 'willow oak', 'sycamore maple',
       'Amur maple', nan, 'hedge maple', 'American elm', 'ash',
       'crab apple', 'silver maple', 'Turkish hazelnut', 'black cherry',
       'eastern redcedar', 'Norway maple', 'tulip-poplar', 'sawtooth oak',
       'Sophora', 'swamp white oak', 'Chinese fringetree',
       'southern magnolia', 'sweetgum', 'Callery pear', 'scarlet oak',
       'Atlantic white cedar', 'black oak', 'Japanese zelkova',
       'white oak', 'Ohio buckeye', 'northern red oak', 'silver linden',
       'pignut hickory', 'Kentucky yellowwood', 'mulberry', 'Douglas-fir',
       'crepe myrtle', 'sassafras', 'spruce', 'Chinese elm',
       'horse chestnut', 'holly', 'littleleaf linden', 'white pine',
       'blackgum', 'cherry', 'Japanese tree lilac', 'hardy rubber tree',
       'green ash', 'English oak', 'white ash', 'golden raintree',
       "Schumard's oak", 'Siberian elm', '

In [29]:
trees[trees.columns[9]].unique()

array(['Acer rubrum', 'Quercus palustris',
       'Gleditsia triacanthos var. inermis', 'Tilia americana',
       'Platanus x acerifolia', 'Ginkgo biloba', 'Quercus phellos',
       'Acer pseudoplatanus', 'Acer ginnala', nan, 'Acer campestre',
       'Ulmus americana', 'Fraxinus', 'Malus', 'Acer saccharinum',
       'Corylus colurna', 'Prunus serotina', 'Juniperus virginiana',
       'Acer platanoides', 'Liriodendron tulipifera',
       'Quercus acutissima', 'Styphnolobium japonicum', 'Quercus bicolor',
       'Chionanthus retusus', 'Magnolia grandiflora',
       'Liquidambar styraciflua', 'Pyrus calleryana', 'Quercus coccinea',
       'Chamaecyparis thyoides', 'Quercus velutina', 'Zelkova serrata',
       'Quercus alba', 'Aesculus glabra', 'Quercus rubra',
       'Tilia tomentosa', 'Carya glabra', 'Cladrastis kentukea', 'Morus',
       'Pseudotsuga menziesii', 'Lagerstroemia', 'Sassafras albidum',
       'Picea', 'Ulmus parvifolia', 'Aesculus hippocastanum', 'Ilex',
       'Tilia cord

Coach said:
    For machine learning purposes, you can get rid of all redundant data. 
    Also, any column that can be reproduced using another one, can be dropped too.

Thus, the_geo can be dropped.

In [35]:
trees[trees.columns[15]].unique()

array(['None', 'Stones', 'MetalGrates', 'RootOther', 'Stones,BranchOther',
       'TrunkLights,BranchLights', 'RootOther,TrunkOther,BranchOther',
       'BranchOther', 'Stones,BranchLights', 'TrunkOther,BranchLights',
       'MetalGrates,TrunkOther', 'TrunkOther', 'Stones,WiresRope',
       'BranchLights', nan, 'WiresRope,BranchOther',
       'RootOther,BranchOther', 'Stones,WiresRope,BranchLights',
       'WiresRope', 'WiresRope,BranchLights', 'Stones,TrunkOther',
       'WiresRope,TrunkLights,BranchLights', 'Stones,RootOther',
       'Stones,BranchLights,BranchOther', 'Stones,RootOther,BranchLights',
       'MetalGrates,BranchOther', 'Stones,RootOther,TrunkOther',
       'RootOther,TrunkOther',
       'Stones,TrunkOther,BranchLights,BranchOther',
       'Stones,TrunkOther,BranchOther', 'RootOther,BranchLights',
       'TrunkOther,BranchOther', 'RootOther,WiresRope,BranchLights',
       'Stones,TrunkOther,BranchLights', 'BranchLights,Sneakers',
       'Stones,RootOther,TrunkOther,Bran